In [1]:
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer, AerSimulator

In [2]:
def nXOR(qc, qubits, output):
    for q in qubits:
        qc.cx(q, output)
    qc.barrier()

def XOR1(qc, a, output):
    qc.cx(a, output)
    qc.barrier()

def get_phase_gate(n):
    size = 2**n
    i = -np.eye(size)
    i[0][0] = 1
    phase_gate = UnitaryGate(i)
    return phase_gate

In [3]:
def diffuser(n):
    qc = QuantumCircuit(n)
    for q in range(n):
        qc.h(q)
    qc.append(get_phase_gate(n), range(n))
    for q in range(n):
        qc.h(q)
    U_s = qc.to_gate()
    U_s.name = "U_s"
    return U_s

In [4]:
def generate_clause_list(A, n):
    clause_list = []
    
    for i in range(n):
        l = []
        for j in range(n):
            if A.item((i, j)) == 1:
                l.append(j)
        clause_list.append(l)
    
    return clause_list

In [5]:
def lights_out_oracle(n, qc, var_qubits, p_qubits, eq_qubits, sol_qubit, c_bits, clause_list):
    
    qc.barrier()
    i = 0
    for clause in clause_list:
        clause = [var_qubits[c] for c in clause]
        nXOR(qc, clause, eq_qubits[i])
        i += 1
    
    for i in range(n):
        XOR1(qc, p_qubits[i], eq_qubits[i])
    qc.x(eq_qubits)
    
    qc.mcx(eq_qubits, sol_qubit)
    
    i = 0
    for clause in clause_list:
        clause = [var_qubits[c] for c in clause]
        nXOR(qc, clause, eq_qubits[i])
        i += 1
    
    for i in range(n):
        XOR1(qc, p_qubits[i], eq_qubits[i])
    qc.x(eq_qubits)
    
    qc.barrier()
    
    qc.append(diffuser(n), range(n))
    

In [6]:
def lights_out_solver(A, p, n, t, init_distribution):
    var_qubits = QuantumRegister(n, name='click')
    p_qubits = QuantumRegister(n, name='initial')
    eq_qubits = QuantumRegister(n, name='output')  
    sol_qubit = QuantumRegister(1, name='solution')
    c_bits = ClassicalRegister(n, name='classical')
    
    clause_list = generate_clause_list(A, n)
    qc = QuantumCircuit(var_qubits, p_qubits, eq_qubits, sol_qubit, c_bits)
    
    # INITIALIZATION
    # - STATE on SOLUTION QUBIT
    qc.x(sol_qubit)
    qc.h(sol_qubit)
    
    # Initial state to solve
    for i in range(n):
        if p[i] == 1:
            qc.x(p_qubits[i])
    
    
    # SUPERPOSITION OF BITSTRING WITH INITIAL DISTRIBUTION
    qc.initialize(init_distribution, var_qubits)
    
    for i in range(t):
        lights_out_oracle(n, qc, var_qubits, p_qubits, eq_qubits, sol_qubit, c_bits, clause_list)
    
    for i in range(n):
        qc.measure(var_qubits[i], c_bits[i])
    
    return qc

In [ ]:
A = np.matrix([
    
])